In [ ]:
import os
import requests
from langchain.agents import initialize_agent, Tool
from langchain.llms import Ollama

In [ ]:
SPOTIFY_ACCESS_TOKEN = os.getenv("SPOTIFY_ACCESS_TOKEN")
if not SPOTIFY_ACCESS_TOKEN:
    raise ValueError("Variável de ambiente SPOTIFY_ACCESS_TOKEN não definida")

In [ ]:
from models.AgentResponseModel import ResponseModel

def pause_spotify_music(_input: str = "") -> ResponseModel:
    """
    Pausa a música no Spotify.
    """
    url = "https://api.spotify.com/v1/me/player/pause"
    headers = {
        "Authorization": f"Bearer {SPOTIFY_ACCESS_TOKEN}"
    }

    response = requests.put(url, headers=headers)

    if response.status_code in [204, 200]:
        return ResponseModel(success=True, response="Música pausada com sucesso no Spotify!")
    elif response.status_code == 403:
        return ResponseModel(success=False, response="Não há player ativo ou permissões insuficientes.")
    else:
        return ResponseModel(
            success=False,
            response=f"Erro ao tentar pausar a música: {response.status_code} - {response.text}"
        )


def play_spotify_music(_input: str = "") -> ResponseModel:
    """
    Inicia a reprodução de música no Spotify.
    """
    url = "https://api.spotify.com/v1/me/player/play"
    headers = {
        "Authorization": f"Bearer {SPOTIFY_ACCESS_TOKEN}"
    }

    response = requests.put(url, headers=headers)

    if response.status_code in [204, 200]:
        return ResponseModel(success=True, response="Música reproduzida com sucesso no Spotify!")
    elif response.status_code == 403:
        return ResponseModel(success=False, response="Não há player ativo ou permissões insuficientes.")
    else:
        return ResponseModel(
            success=False,
            response=f"Erro ao tentar iniciar a música: {response.status_code} - {response.text}"
        )


In [ ]:
play_spotify_music()

In [ ]:
from langchain.agents import Tool

tools = [
    Tool(
        name="PauseSpotifyMusic",
        func=pause_spotify_music,
        description="Pausa a música atual no Spotify"
    ),
    Tool(
        name="PlaySpotifyMusic",
        func=play_spotify_music,
        description="Inicia a reprodução da música atual no Spotify"
    )
]


In [ ]:
llm = Ollama(model="llama3:70b", temperature=0.7)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
parser = JsonOutputParser(pydantic_object=ResponseModel)

In [ ]:
from langchain.prompts import PromptTemplate

def build_main_prompt_agent(instruction_parser: JsonOutputParser):
    prompt_template = PromptTemplate(
        template=""" 
            Você é um assistente que controla música no Spotify: \n
            
            Responda sempre no formato JSON:\n
            {format_instructions}
            
            Em relação a resposta:
            
            - Certifique-se de manter o formato JSON corretamente de acordo com a instrução.
            - Responda apenas o JSON, e nenhum texto a mais.
            - A resposta deve ser **apenas o JSON puro**, sem introduções, explicações ou formatações adicionais.
            
            entrada do usuário:
            {input}
            """,
        input_variables=["input"],
        partial_variables={
            "format_instructions": instruction_parser.get_format_instructions()
        }
    )

    return prompt_template

In [ ]:
prompt_template = build_main_prompt_agent(parser)

In [ ]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=1,
    return_intermediate_steps=True
)

result = agent.invoke({"input": "Play a música no Spotify"})


output_final = result["output"]
intermediate_steps = result["intermediate_steps"]

first_observation = intermediate_steps[0][1]



In [ ]:
import re

def format_json_output(response: str, parser: any) -> str:
    match = re.search(r"{.*}", response, re.DOTALL)
    json_str = match.group(0)
    persed_response = parser.parse(json_str)
    return persed_response


In [ ]:
from models.AgentResponseModel import ResponseModel

parsed_response = ResponseModel.model_validate(first_observation)


In [ ]:
print(parsed_response.response)

In [ ]:
from agents.spotify.spotify_agent import AgentSpotify

if __name__ == "__main__":
    agent = AgentSpotify()

    user_input = "Retome a música no Spotify"
    response = agent.run(user_input)
    print(response)


In [ ]:
from agents.main.main_agent import AgentMain


if __name__ == "__main__":
    agente_principal = AgentMain()

    comando_teste = "Retome a música"

    resposta = agente_principal.run(comando_teste)

    print("Resposta do agente principal:")
    print(resposta)




> Entering new AgentExecutor chain...
Let's think about this...

Action: pause_song
Action Input: None (no specific song or action needed, just pause the current music)
Observation: Música pausada com sucesso.
Thought:

> Finished chain.
Resposta do agente principal:
Música pausada com sucesso.


In [ ]:
from agents.spotify.resources.resource_play_music import play_pause_music


play_pause_music("play")